In [109]:
from bs4 import BeautifulSoup as BS
from selenium import webdriver
from functools import reduce
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
import pandas as pd
import time

In [310]:
class Parser:
    def __init__(self):
        self.url = "https://schedule.kpi.ua/?groupId=9bd9f12a-5083-4e2f-9e21-18445e79883e"
        self.lesson_classes = ['sc-dLMFU', 'jHsfpU', 'sc-dcJsrY',  'hoEsOx', 'sc-eDPEul', 'dbZAma']
        self.time = ['8:30', '10:25', '12:20', '14:15', '16;10', '18;30']
        self.days = ['Mon', 'Tus', 'Wed', 'Thu', 'Fri', 'Sat']
        

    def get_group_page(self, group_name: str, week: int):
        driver = webdriver.Chrome()
        driver.maximize_window()
        driver.get(self.url)

        input_elem = WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.ID, "react-select-3-input")))
        input_elem.click()
        input_elem.send_keys(group_name)
        time.sleep(0.5)
        input_elem.send_keys(Keys.ENTER)
        if week==2:
            WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.CLASS_NAME, "hnMYvq"))).click()
        else:
            WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.CLASS_NAME, "eBQRSr"))).click()
        WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.CLASS_NAME, "uQJdd"))).click()
        time.sleep(0.5)
        r = driver.page_source
        driver.quit()
        return r

    def get_group_schedule(self, group_name: str, week: int):
        page = self.get_group_page(group_name, week)
        soup = BS(page, "html.parser")
        divs = soup.findAll('div')
        
        blocks = []
        for div in divs:
            if 'class' in div.attrs:
                parse=0
                for wclass in to_parse:
                    if wclass in div['class']:
                        parse=1
                        break
                if parse:
                    blocks.append(div)

        week = dict()
        
        for i in range(6):
            for j in range(6):
                if i==0:
                    week[self.days[j]]=dict()
                div = blocks[i*6+j]
                if 'jHsfpU' in div['class']:
                    lesson = None
                else:
                    info = div.findAll('div')
                    print(div)
                    print('-------------------------------')
                    lesson = dict()
                    for elem in info:
                        if 'class' in elem.attrs:
                            if 'hXteRa' in elem['class']:
                                lesson['type'] = elem.text
                            if 'bMGeNQ' in elem['class']:
                                lesson['type'] = elem.text
                            if 'hbZTEb' in elem['class']:
                                lesson['type'] = elem.text
                            if 'fjzqii' in elem['class']:
                                lesson['discription'] = elem.text
                            if 'cmzwJF' in elem['class']:
                                lesson['lector'] = elem.text
                week[self.days[j]][self.time[i]] = lesson

        return week
        

In [311]:
data

{'Mon': {'8:30': {'type': 'Лек',
   'discription': 'Фізика. Частина 1. Механіка. Молекулярна фізика',
   'lector': 'Чурсанова Марина Валеріївна'},
  '10:25': {'type': 'Лек',
   'discription': 'Вища математика. Частина 1. Диференційні числення. Аналітична геометрія. Лінійна алгебра',
   'lector': 'Коновалова Наталія Романівна'},
  '12:20': {'type': 'Лек', 'discription': 'Історія науки і техніки'},
  '14:15': {'type': 'Лек',
   'discription': 'Інженерні основи авіації і космонавтики',
   'lector': 'Кабанячий Володимир Володимирович'},
  '16;10': None,
  '18;30': None},
 'Tus': {'8:30': {'type': 'Лек',
   'discription': 'Інформаційні технології та загальні методи розробки прикладного програмного забезпечення. Частина 1. Інформаційні технології',
   'lector': 'Борисов Віктор Васильович'},
  '10:25': {'type': 'Лек',
   'discription': 'Нарисна геометрія',
   'lector': 'Ванін Володимир Володимирович'},
  '12:20': {'type': 'Лек',
   'discription': 'Хімія',
   'lector': 'Шпак Арсеній Євгенович'

In [312]:
p = Parser()
data = p.get_group_schedule('AA-31 (IAT)', 1) #'БТ-в27ф (ФБМІ)'

<div class="sc-dcJsrY hoEsOx"><div><div class="sc-aXZVg sc-jEACwC knHWgd jeiDIp"><div class="sc-iGgWBj sc-imWYAI djcdjy hXteRa">Лек</div></div><div class="sc-jXbUNg fjzqii">Фізика. Частина 1. Механіка. Молекулярна фізика</div><div class="sc-aXZVg sc-dhKdcB sc-kpDqfm knHWgd iuSRFk cmzwJF"><img alt="teacher" class="sc-gEvEer hIFHmP" src="/static/media/teacher.b868ff5585b1165a8f3f16b897d7eff6.svg"/><a class="sc-fqkvVR eIJnWA" href="/lecturers?lecturerId=5bd5be19-78fd-4fe3-a3b4-3489ba1f9321">Чурсанова Марина Валеріївна</a></div><div class="sc-aXZVg sc-dhKdcB sc-cwHptR knHWgd iuSRFk dtpzom"><div><a class="sc-fqkvVR eIJnWA" href="/?groupId="> </a></div></div></div></div>
-------------------------------
<div class="sc-dcJsrY hoEsOx"><div><div class="sc-aXZVg sc-jEACwC knHWgd jeiDIp"><div class="sc-iGgWBj sc-imWYAI djcdjy hXteRa">Лек</div></div><div class="sc-jXbUNg fjzqii">Інформаційні технології та загальні методи розробки прикладного програмного забезпечення. Частина 1. Інформаційні техноло

In [263]:
info = data['Wed']['10:25'].findAll('div')


In [265]:
lesson

{'type': 'Лек',
 'discription': 'Проблеми розробки та випробування біофармацевтичних препаратів',
 'lector': 'Галкін Олександр Юрійович'}

In [234]:
def render_page(url, group_name, week):
    driver = webdriver.Chrome()
    driver.maximize_window()
    driver.get(url)
    input_elem = WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.ID, "react-select-3-input")))
    input_elem.click()
    input_elem.send_keys(group_name)
    time.sleep(0.5)
    input_elem.send_keys(Keys.ENTER)
    time.sleep(0.5)
    if week==2:
        WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.CLASS_NAME, "hnMYvq"))).click()
    else:
        WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.CLASS_NAME, "eBQRSr"))).click()
    time.sleep(3)
    r = driver.page_source
    driver.quit()
    return r

In [235]:
r = render_page("https://schedule.kpi.ua/?groupId=9bd9f12a-5083-4e2f-9e21-18445e79883e", final_dd[200], 1)
soup = BS(r, "html.parser")
mydivs = soup.findAll('div')
to_parse = ['sc-dLMFU', 'jHsfpU', 'sc-dcJsrY',  'hoEsOx', 'sc-eDPEul', 'dbZAma']
parsed = []
for div in mydivs:
    if 'class' in div.attrs:
        parse=0
        for wclass in to_parse:
            if wclass in div['class']:
                parse=1
                break
        if parse:
            print(div['class'])
            parsed.append(div)
            pass


['sc-dcJsrY', 'hoEsOx']
['sc-eDPEul', 'dbZAma']
['sc-dcJsrY', 'hoEsOx']
['sc-dcJsrY', 'hoEsOx']
['sc-dLMFU', 'jHsfpU']
['sc-dcJsrY', 'hoEsOx']
['sc-dcJsrY', 'hoEsOx']
['sc-dcJsrY', 'hoEsOx']
['sc-dcJsrY', 'hoEsOx']
['sc-dcJsrY', 'hoEsOx']
['sc-dLMFU', 'jHsfpU']
['sc-dcJsrY', 'hoEsOx']
['sc-dcJsrY', 'hoEsOx']
['sc-dcJsrY', 'hoEsOx']
['sc-dLMFU', 'jHsfpU']
['sc-dcJsrY', 'hoEsOx']
['sc-dLMFU', 'jHsfpU']
['sc-dLMFU', 'jHsfpU']
['sc-dcJsrY', 'hoEsOx']
['sc-dcJsrY', 'hoEsOx']
['sc-dcJsrY', 'hoEsOx']
['sc-dcJsrY', 'hoEsOx']
['sc-dLMFU', 'jHsfpU']
['sc-dLMFU', 'jHsfpU']
['sc-dcJsrY', 'hoEsOx']
['sc-dcJsrY', 'hoEsOx']
['sc-dLMFU', 'jHsfpU']
['sc-dcJsrY', 'hoEsOx']
['sc-dLMFU', 'jHsfpU']
['sc-dLMFU', 'jHsfpU']
['sc-dLMFU', 'jHsfpU']
['sc-dLMFU', 'jHsfpU']
['sc-dLMFU', 'jHsfpU']
['sc-dLMFU', 'jHsfpU']
['sc-dLMFU', 'jHsfpU']
['sc-dLMFU', 'jHsfpU']


In [214]:
import json
#with open("groups.json", "w") as outfile:
    #for elem in final_dd:
        #outfile.write(elem+'\n')
        #pass

In [217]:
r = render_page("https://schedule.kpi.ua/?groupId=9bd9f12a-5083-4e2f-9e21-18445e79883e", final_dd[3]) #fast-option-selected

In [208]:
len(final_dd)

1450

In [180]:
group

'АА-31 (ІАТ)'

In [46]:
import numpy as np

In [53]:
time = ['8:30', '10:25', '12:20', '14:15', '16;10', '18;30']
days = ['Mon', 'Tus', 'Wed', 'Thu', 'Fri', 'Sat']
week = dict()

In [54]:
len(parsed)

36

In [55]:
for i in range(6):
    for j in range(6):
        if i==0:
            week[days[j]]=dict()
        div = parsed[i*6+j]
        week[days[j]][time[i]] = div['class']

In [56]:
week

{'Mon': {'8:30': ['sc-dcJsrY', 'hoEsOx'],
  '10:25': ['sc-dcJsrY', 'hoEsOx'],
  '12:20': ['sc-dcJsrY', 'hoEsOx'],
  '14:15': ['sc-dcJsrY', 'hoEsOx'],
  '16;10': ['sc-dLMFU', 'jHsfpU'],
  '18;30': ['sc-dLMFU', 'jHsfpU']},
 'Tus': {'8:30': ['sc-dcJsrY', 'hoEsOx'],
  '10:25': ['sc-dcJsrY', 'hoEsOx'],
  '12:20': ['sc-dcJsrY', 'hoEsOx'],
  '14:15': ['sc-dLMFU', 'jHsfpU'],
  '16;10': ['sc-dLMFU', 'jHsfpU'],
  '18;30': ['sc-dLMFU', 'jHsfpU']},
 'Wed': {'8:30': ['sc-dcJsrY', 'hoEsOx'],
  '10:25': ['sc-dcJsrY', 'hoEsOx'],
  '12:20': ['sc-dcJsrY', 'hoEsOx'],
  '14:15': ['sc-dLMFU', 'jHsfpU'],
  '16;10': ['sc-dLMFU', 'jHsfpU'],
  '18;30': ['sc-dLMFU', 'jHsfpU']},
 'Thu': {'8:30': ['sc-dcJsrY', 'hoEsOx'],
  '10:25': ['sc-eDPEul', 'dbZAma'],
  '12:20': ['sc-dcJsrY', 'hoEsOx'],
  '14:15': ['sc-dLMFU', 'jHsfpU'],
  '16;10': ['sc-dLMFU', 'jHsfpU'],
  '18;30': ['sc-dLMFU', 'jHsfpU']},
 'Fri': {'8:30': ['sc-dLMFU', 'jHsfpU'],
  '10:25': ['sc-dLMFU', 'jHsfpU'],
  '12:20': ['sc-dcJsrY', 'hoEsOx'],
  '14:1